# PackedSequence, PaddedSequence

[링크: PackedSequence에 대한 PyTorch 공식 문서](https://pytorch.org/docs/stable/nn.html#packedsequence)

이 튜토리얼에서는 RNN / LSTM 계열의 모델에서 sequence batch를 잘 활용할 수 있는 `PackedSequence` 와 `PaddedSequence`를 만드는 법을 배워보겠습니다.

PyTorch 라이브러리 안에는 다음 4가지 함수들이 주어집니다.

`pad_sequence`, `pack_sequence`, `pack_padded_sequence`, `pad_packed_sequence`

In [1]:
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

In [2]:
# example data generating. batch size = 5, longest seq = 13
# Random word from random word generator
data = ['hello world',
        'midnight',
        'calculation',
        'path',
        'short circuit']

# Make dictionary
# 다음과 같이 해석한다 : char (for seq in (data for char in seq))
char_set = ['<pad>'] + list(set(char for seq in data for char in seq)) # Get all characters and include pad token 
char2idx = {char: idx for idx, char in enumerate(char_set)} # Constuct character to index dictionary
print('char_set:', char_set)
print('char_set length:', len(char_set))

char_set: ['<pad>', 'w', 'd', 'i', 'a', 'm', 'e', 'h', 'o', 's', 'n', 'g', 'u', 'c', 'p', ' ', 'l', 'r', 't']
char_set length: 19


In [3]:
# Convert character to index and make list of tensors
X = [torch.LongTensor([char2idx[char] for char in seq]) for seq in data]

# Check converted result
for sequence in X:
    print(sequence)
print(X)

tensor([ 7,  6, 16, 16,  8, 15,  1,  8, 17, 16,  2])
tensor([ 5,  3,  2, 10,  3, 11,  7, 18])
tensor([13,  4, 16, 13, 12, 16,  4, 18,  3,  8, 10])
tensor([14,  4, 18,  7])
tensor([ 9,  7,  8, 17, 18, 15, 13,  3, 17, 13, 12,  3, 18])
[tensor([ 7,  6, 16, 16,  8, 15,  1,  8, 17, 16,  2]), tensor([ 5,  3,  2, 10,  3, 11,  7, 18]), tensor([13,  4, 16, 13, 12, 16,  4, 18,  3,  8, 10]), tensor([14,  4, 18,  7]), tensor([ 9,  7,  8, 17, 18, 15, 13,  3, 17, 13, 12,  3, 18])]


In [4]:
# 위의 tensor는 모두 그 size가 제각각이다.
# Make length tensor (will be used later in 'pack_padded_sequence' function)
lengths = [len(seq) for seq in X]
print('lengths:', lengths)

lengths: [11, 8, 11, 4, 13]


하나의 batch로 만들어주기 위해서 일반적으로 제일 긴 sequence 길이에 맞춰 뒷부분에 padding을 추가(일반적으로 많이 쓰이는 Padding 방식)

PyTorch에서는 `PackedSequence`라는 것을 쓰면 padding 없이도 정확히 필요한 부분까지만 병렬 계산을 할 수도 있다.

## `pad_sequence` 함수를 이용하여 PaddedSequence (그냥 Tensor) 만들기

따로 PaddedSequence라는 class는 존재하지 않고, PaddedSequence는 sequence중에서 가장 긴 sequence와 길이를 맞추어주기 위해 padding을 추가한 일반적인 **Tensor**를 말한다.

이때, `pad_sequence`라는 PyTorch 기본 라이브러리 함수를 이용한다. input이 **Tensor들의 list**이다.

list 안에 있는 각각의 Tensor들의 shape가 `(?, a, b, ...)` 라고 할때, (여기서 ?는 각각 다른 sequence length 입니다.)

input Tensor의 shape = `(sequence length, a, b, ...)`

output Tensor shape = `(longest sequence length in the batch, batch_size, a, b, ...)`

`batch_first=True`이면 `(batch_size, longest sequence length in the batch, a, b, ...)` shape를 가지는 Tensor가 리턴된다.

`padding_value=42`와 같이 파라미터를 지정해주면, padding하는 값도 정해진다.

In [5]:
# Make a Tensor of shape (Batch x Maximum_Sequence_Length)
padded_sequence = pad_sequence(X, batch_first=True) # X is now padded sequence
print(padded_sequence)
print(padded_sequence.shape)

padded_sequence = pad_sequence(X) # X is now padded sequence
print(padded_sequence)
print(padded_sequence.shape)

tensor([[ 7,  6, 16, 16,  8, 15,  1,  8, 17, 16,  2,  0,  0],
        [ 5,  3,  2, 10,  3, 11,  7, 18,  0,  0,  0,  0,  0],
        [13,  4, 16, 13, 12, 16,  4, 18,  3,  8, 10,  0,  0],
        [14,  4, 18,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 9,  7,  8, 17, 18, 15, 13,  3, 17, 13, 12,  3, 18]])
torch.Size([5, 13])
tensor([[ 7,  5, 13, 14,  9],
        [ 6,  3,  4,  4,  7],
        [16,  2, 16, 18,  8],
        [16, 10, 13,  7, 17],
        [ 8,  3, 12,  0, 18],
        [15, 11, 16,  0, 15],
        [ 1,  7,  4,  0, 13],
        [ 8, 18, 18,  0,  3],
        [17,  0,  3,  0, 17],
        [16,  0,  8,  0, 13],
        [ 2,  0, 10,  0, 12],
        [ 0,  0,  0,  0,  3],
        [ 0,  0,  0,  0, 18]])
torch.Size([13, 5])


# `pack_sequence` 함수를 이용하여 PackedSequence 만들기

padding을 추가하지 않고 정확히 주어진 sequence 길이까지만 모델이 연산을 하게끔 만드는 PyTorch의 자료구조이다.

주어지는 input (list of Tensor)는 길이에 따른 내림차순으로 정렬이 되어있어야 한다.

sorted 함수에 대해서는 다음 링크를 확인

https://blockdmask.tistory.com/466

In [6]:
# Sort by descending lengths
sorted_idx = sorted(range(len(lengths)), key=lengths.__getitem__, reverse=True) # docu 참고
sorted_X = [X[idx] for idx in sorted_idx] # sorting한 결과

# Check converted result
for sequence in sorted_X:
    print(sequence)

tensor([ 9,  7,  8, 17, 18, 15, 13,  3, 17, 13, 12,  3, 18])
tensor([ 7,  6, 16, 16,  8, 15,  1,  8, 17, 16,  2])
tensor([13,  4, 16, 13, 12, 16,  4, 18,  3,  8, 10])
tensor([ 5,  3,  2, 10,  3, 11,  7, 18])
tensor([14,  4, 18,  7])


In [16]:
packed_sequence = pack_sequence(sorted_X)
print(packed_sequence) # 여기서의 batch_size는 각 sequence를 처리하는 step 마다의 batch를 의미한다. 함수값으로 입력되는 sequence의 length가 아니라!
print(packed_sequence.data.shape)

PackedSequence(data=tensor([ 9,  7, 13,  5, 14,  7,  6,  4,  3,  4,  8, 16, 16,  2, 18, 17, 16, 13,
        10,  7, 18,  8, 12,  3, 15, 15, 16, 11, 13,  1,  4,  7,  3,  8, 18, 18,
        17, 17,  3, 13, 16,  8, 12,  2, 10,  3, 18]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]), sorted_indices=None, unsorted_indices=None)
torch.Size([47])


# RNN에 투입하기

In [8]:
# character의 index를 one-hot character embedding한 값을 RNN의 input으로 넣어준다.
# one-hot embedding using PaddedSequence
eye = torch.eye(len(char_set)) # Identity matrix of shape (len(char_set), len(char_set))
embedded_tensor = eye[padded_sequence]
print(embedded_tensor.shape) # shape: (Batch_size, max_sequence_length, number_of_input_tokens)

torch.Size([13, 5, 19])


In [9]:
# one-hot embedding using PackedSequence
embedded_packed_seq = pack_sequence([eye[X[idx]] for idx in sorted_idx])
print(embedded_packed_seq.data.shape)

torch.Size([47, 19])


# RNN 모델에 투입

In [10]:
# declare RNN
rnn = torch.nn.RNN(input_size=len(char_set), hidden_size=30, batch_first=True)

`PaddedSequence`를 이용하여 RNN에 넣어봅시다.

In [11]:
rnn_output, hidden = rnn(embedded_tensor)
print(rnn_output.shape) # shape: (batch_size, max_seq_length, hidden_size)
print(hidden.shape)     # shape: (num_layers * num_directions, batch_size, hidden_size)

torch.Size([13, 5, 30])
torch.Size([1, 13, 30])


`PackedSequence`를 이용하여 RNN에 넣어봅시다.

In [12]:
rnn_output, hidden = rnn(embedded_packed_seq)
print(rnn_output.data.shape)
print(hidden.data.shape)

torch.Size([47, 30])
torch.Size([1, 5, 30])


In [13]:
#`pad_packed_sequence` -> (Tensor, list_of_lengths)

unpacked_sequence, seq_lengths = pad_packed_sequence(embedded_packed_seq, batch_first=True)
print(unpacked_sequence.shape)
print(seq_lengths)

torch.Size([5, 13, 19])
tensor([13, 11, 11,  8,  4])


In [14]:
# `pack_padded_sequence` : sequence 길이반드시 input, input sequence가 내림차순 정렬 되어있어야함

# sorting
embedded_padded_sequence = eye[pad_sequence(sorted_X, batch_first=True)]
print(embedded_padded_sequence.shape)

# pack_padded_sequence
sorted_lengths = sorted(lengths, reverse=True)
new_packed_sequence = pack_padded_sequence(embedded_padded_sequence, sorted_lengths, batch_first=True)
print(sorted_lengths)
print(new_packed_sequence.data.shape)
print(new_packed_sequence.batch_sizes) # 여기서의 batch_size는 각 sequence를 처리하는 step 마다의 batch를 의미한다. 함수값으로 입력되는 sequence의 length가 아니라!

torch.Size([5, 13, 19])
[13, 11, 11, 8, 4]
torch.Size([47, 19])
tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1])
